CNN for text classification
=============
<span style="color: lightsteelblue;">Deep Learning</span>

The goal of this notebook is to train convolutional neural network over text extracted from public articles at [Lenta.ru](https://lenta.ru) for text classification purposes.

**We will use**
- pretrained embeddings from previous word2vec CBoW model
- also (option 2) train embeddings from scratch
- 3 different filter sizes
- relu activation
- max-pooling
- and a dropout

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
def read_data(folder):
  data = {}
  data_files = os.listdir(folder)
  for data_file in data_files:
    with open(os.path.join(folder,data_file)) as f:
      articles = []
      for line in f:
        articles.append(line)
    data[data_file] = articles

  return data

data = read_data('articles')

for label in data:
  excerpt = data[label][0][:175] + '...'
  print('Number of %s articles is %d. For example: \n%s' % (label, len(data[label]), excerpt))

Number of business articles is 3. For example: 
Индийский слон Кофейный рынок лихорадит. К плохим новостям из Южной Америки, гуляющим уже несколько недель, на днях добавились тревожные звонки из Индии, которая является веду...
Number of culture articles is 6. For example: 
Если погуглить рекомендательные подборки семейных саг, то на разных литературных сайтах в списке из условных 10 позиций непременно окажется «Сага о Форсайтах» и, надо думать, ...
Number of sport articles is 4. For example: 
Сами себе чемпионы Среди всех участников чемпионата мира на этот раз было лишь 19 обладателей паспорта в бордовой корочке. Хотя формально ни одного россиянина на турнире не бы...


Let's now prepare Convolutional Neural network. We will wrap it up in `ArticlesClassifier` class to make it reuseable:

In [70]:
class ArticlesClassifier(object):
  def __init__(self, sequence_length, num_labels, vocab_size,
               embedding_size, filter_sizes, filter_out_channels):
    # Placeholders for input, output and dropout
    self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
    self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
    self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

    # Embeddings layer
    with tf.device('/cpu:0'), tf.name_scope("embedding"):
      self.embeddings = tf.Variable(
          tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0), name="W")
      self.embedded_chars = tf.nn.embedding_lookup(W, self.input_x)
      self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

    # Convolution and pooling
    pooled = []
    for i, filter_size in enumerate(filter_sizes):
      with tf.name_scope("conv-maxpool-%s" % filter_size):
        # Convolution layer
        filter_shape = [filter_size, embedding_size, 1, filter_out_channels]
        strides = [1, 1, 1, 1]
        W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name='W')
        b = tf.Variable(tf.constant(0.1, [filter_out_channels]), name='b')
        conv = tf.nn.conv2d(self.embedded_chars_expanded, W, strides, 'VALID', name='conv')
        # Bias and non-linearity
        net = tf.nn.bias_add(conv, b)
        h = tf.nn.relu(net, name="h")
        # Pooling
        h_pooled = tf.nn.max_pool(h, [1, sequence_length - filter_size + 1, 1, 1], 'VALID', 'pool')
        pooled.append(h_pooled)

    total_channels = len(filter_sizes) * filter_out_channels
    self.h_pool = tf.concat(3, pooled)
    self.h_pool_flat = tf.reshape(self.h_pool, [-1, total_channels])
 
    # Dropout layer
    with tf.name_scope("dropout"):
      self.h_drop = tf.nn.dropout(self.h_pool_flat, dropout_keep_prob)

    # Output layer (Scores and predictions)
    W = tf.Variable(tf.truncated_normal([total_channels, num_classes], stddev=0.1), name='W')
    b = tf.Variable(tf.constant(0.1, [num_classes]), name='b')
    self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
    self.predictions = tf.argmax(self.scores, 1, name="predictions")

    # Loss and accuracy
    with tf.name_scope("loss"):
      losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
      self.loss   = tf.reduce_mean(losses)

  def train():
    optimizer = tf.AdamOptimizer(1e-4)
    optimizer.train(self.loss, feed_dict: feed_dict)
    



str
 anarchism originated as a term of abuse first use
str
I go to buy some chocolate
str
I go to buy some milk
list
{'the': 1, 'of': 2, 'and': 3, 'one': 4, 'in': 5, 'a': 6, 'to': 7, 'zero': 8, 'nine': 9, 'two': 10}


Function to generate a training batch for the RNN model.

In [ ]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

In [0]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple RNN Model.

In [0]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [ ]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
